Для проверки модели произведу идентификацию по своему голосу. Для этого необходимо обучить модель на данных, включающих мой голос, а также применить фильтрацию по порогу, который определяет, есть тот или иной голос в рассматрвиаемой базе голосов.

Тестировать буду наилучшую модель, а именно catboost с подобранными параметрами

In [1]:
from my_funcs import get_dataframe, get_model_and_params, get_feature
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from scipy.spatial.distance import cosine

In [2]:
df = pd.read_pickle('experiments/crowd_train_all_data_embedded.pkl')

In [3]:
df_t = get_dataframe(df, 100, 10)
df_t.head()

354


,audio_path,source_id,audio_feature
0,crowd_train\wavs/cb9733d3521a52dd7a578503dc982...,02d58cb47f02f8884aaa45b0f7dd7714,"[0.36070194840431213, 0.36269721388816833, 0.3..."
1,crowd_train\wavs/40dddbe3d8c0bcb4fd8b3c933f660...,02d58cb47f02f8884aaa45b0f7dd7714,"[0.38155198097229004, 0.4070906341075897, 0.44..."
2,crowd_train\wavs/19a6f9ce1902ffb0f45c05f5e621e...,02d58cb47f02f8884aaa45b0f7dd7714,"[0.36854031682014465, 0.4035727083683014, 0.44..."
3,crowd_train\wavs/d9050a9ec2523b61435ad2a2b4aca...,02d58cb47f02f8884aaa45b0f7dd7714,"[0.33979716897010803, 0.35216024518013, 0.3810..."
4,crowd_train\wavs/a931c958489a78e12956ebee1e13a...,02d58cb47f02f8884aaa45b0f7dd7714,"[0.32755956053733826, 0.3753350079059601, 0.39..."


In [4]:
df_me = pd.read_parquet('evaluation/eval_df.pqt')
df_me.head()

,audio_path,source_id,audio_feature
0,maria\Record_0.wav,maria,"[0.22580918669700623, 0.23982486128807068, 0.1..."
1,maria\Record_1.wav,maria,"[0.2748687267303467, 0.2674121558666229, 0.265..."
2,maria\Record_2.wav,maria,"[0.21786867082118988, 0.25627416372299194, 0.3..."
3,maria\Record_3.wav,maria,"[0.3032059073448181, 0.2491905242204666, 0.236..."
4,maria\Record_4.wav,maria,"[0.17937365174293518, 0.1685197949409485, 0.26..."


In [5]:
eval_df = pd.concat([df_t, df_me]).reset_index(drop=True)

In [6]:
X = np.vstack(eval_df.audio_feature.to_numpy()).reshape(eval_df.shape[0] , -1)
le = LabelEncoder()
y = pd.DataFrame(le.fit_transform(eval_df.source_id))

X.shape, y.shape

((3560, 498), (3560, 1))

In [7]:
le.transform(['me', 'maria']) #добавленные мной классы

array([355, 354])

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)

Мои записи голоса, которые не использовались для обучения

In [9]:
eval_df.iloc[list(set(eval_df[eval_df.source_id == 'me'].index) & set(y_val.index))].sort_index()

,audio_path,source_id,audio_feature
3551,me\Record_1.wav,me,"[0.3570057451725006, 0.391750305891037, 0.2994..."
3553,me\Record_3.wav,me,"[0.333279013633728, 0.3944181501865387, 0.3920..."
3555,me\Record_5.wav,me,"[0.32722488045692444, 0.40390798449516296, 0.2..."
3556,me\Record_6.wav,me,"[0.3516620993614197, 0.39537549018859863, 0.39..."
3559,me\Record_9.wav,me,"[0.4068811237812042, 0.4601544439792633, 0.555..."


Дообучение модели на + 2 новых классах

In [10]:
import catboost
import json

already_exists = True

if not already_exists:
    pass
    # код, использовавшийся для обучения вынесен в get_evaluation.ipynb
    # из-за громозкости
else:
    model_cb = catboost.CatBoostClassifier()
    model_cb.load_model('cb_model')
    model_name = 'catboost tuned'

    with open(f'cross_validation_scores/{model_name}_threshold.json', 'r') as f:
        d = json.load(f)
    

In [11]:
import gradio as gr

def authorize(audio):
    features = get_feature(audio)
    probs = model_cb.predict_proba(features)
    pred_class = le.inverse_transform([np.argmax(probs)])[0]
    max_prob = np.max(probs)

    result = f'Предсказан класс "{pred_class}" с вероятностью {round(max_prob*100, 2)}%'

    c = 0
    train_part_df = eval_df.iloc[
        list(set(eval_df[eval_df.source_id == pred_class].index) - set(y_val.index))
    ]
    for cos_dist_true in train_part_df.audio_feature:
        if cosine(cos_dist_true, features) <= d['tpr@fpr_threshold']:
            c += 1

    if c >= 3:
        result += '\n\nЧеловек присутствует в датасете исходя из косинусного расстояния'
    else:
        result += (
            '\n\nЧеловек НЕ присутствует в датасете исходя из косинусного расстояния'
        )
    if max_prob >= d['threshold']:
        result += '\nЧеловек присутствует в датасете исходя из разделяющего порога вероятности'
    else:
        result += '\nЧеловек НЕ присутствует в датасете исходя из разделяющего порога вероятности'

    return result


with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            audio = gr.Audio(sources=["upload", "microphone"], type="filepath")
        with gr.Column():
            textbox = gr.Textbox()

    button = gr.Button("Отправить")
    button.click(fn=authorize, inputs=[audio], outputs=[textbox])

demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\Users\Никита\AppData\Roaming\Python\Python39\site-packages\uvicorn\protocols\http\httptools_impl.py", line 419, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "C:\Users\Никита\AppData\Roaming\Python\Python39\site-packages\uvicorn\middleware\proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "C:\Users\Никита\AppData\Roaming\Python\Python39\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "C:\Users\Никита\AppData\Roaming\Python\Python39\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "C:\Users\Никита\AppData\Roaming\Python\Python39\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "C:\Users\Никита\AppData\Roaming\Python\Python39\site-packages\st

In [ ]:
import IPython.display as ipd

audio_file = df_t[df_t['source_id'] == '28245faacb3c27e84322dc9547f667ff'].audio_path.iloc[2]
ipd.Audio(audio_file)